This notebook is to make a video grid

In [1]:
import os
from pathlib import Path
import torch
import json
from torch import Tensor
from jaxtyping import Float, UInt8
from io import BytesIO
import torchvision.transforms as tf
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.image as mpimg
import cv2
import numpy as np
import glob

In [2]:
# Directories of the real data
video_index_path = "/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/assets/evaluation_index_re10k_video.json"
root = Path('/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/datasets/re10k_full')
# root = Path('/teamspace/studios/re10k_data_prep/re10k')
data_stages = ["test"]

# Directories of the generated data such that the folders within are the room keys
# gen_root = Path('../output/images/test_re10k')
# gen_root = Path('/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k')
gen_root = Path('../outputs/re10k_test_hard_more/re10k')

In [3]:
# number of videos you want to produce
num_vid = 5

The code below is to save the real images from the torch files into a folder

In [7]:
merged_index = {}
path_to_torch = []

for data_stage in data_stages:
    # Load the root's index.
    # p = Path("/teamspace/studios/re10k_data_prep/re10k/test/index.json")
    with (root / data_stage / "index.json").open("r") as f:
    # with (p).open("r") as f:
        index = json.load(f)    
    for k, v in index.items():
        path = Path(root / data_stage / v)
        index[k] = path
        if path not in path_to_torch:
            path_to_torch.append(path)
    # index = {k: Path(root / data_stage / v) for k, v in index.items()}

    # The constituent datasets should have unique keys.
    assert not (set(merged_index.keys()) & set(index.keys()))

    # Merge the root's index into the main index.
    merged_index = {**merged_index, **index}

print(merged_index)
print(f"#rooms: {len(merged_index.keys())}")
print(path_to_torch)
print(f"#torch files: {len(path_to_torch)}")
# ../datasets/re10k/test

FileNotFoundError: [Errno 2] No such file or directory: '/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/datasets/re10k_full/test/index.json'

In [6]:
tensor = tf.ToTensor()

def convert_images(images: list[UInt8[Tensor, "..."]],) -> Float[Tensor, "batch 3 height width"]:
    torch_images = []
    for image in images:
        image = Image.open(BytesIO(image.numpy().tobytes()))
        torch_images.append(tensor(image))
    return torch.stack(torch_images)

def center_crop(
    images: Float[Tensor, "*#batch c h w"],
    shape: tuple[int, int],
) -> tuple[
    Float[Tensor, "*#batch c h_out w_out"],  # updated images
]:
    *_, h_in, w_in = images.shape
    h_out, w_out = shape
    # print(h_in, w_in, h_out, w_out)

    # Note that odd input dimensions induce half-pixel misalignments.
    row = (h_in - h_out) // 2
    col = (w_in - w_out) // 2

    # Center-crop the image.
    images = images[..., :, row : row + h_out, col : col + w_out]

    return images

def save_images(path_to_torch, out_dir):
    out_dir = Path(out_dir)
    out_dir.mkdir(exist_ok=True, parents=True)

    # num_rooms = sum(len(torch.load(path)) for path in path_to_torch)
    # print(f"Total amount of rooms: {num_rooms}")

    n = 0

    keys = []

    for path in path_to_torch:
        chunk = torch.load(path)
        for room in tqdm(chunk, desc="Processing rooms", unit="room"):

            print(f"n: {n}")

            if n >= num_vid:
                break

            key = room["key"]
            keys.append(key)
            print(key)

            images = room["images"]
            context_images = convert_images(images)

            room_path = out_dir / Path(key)
            room_path.mkdir(exist_ok=True, parents=True)

            for i, image in enumerate(context_images):
                # img = center_crop(image, (256, 256))
                img = center_crop(image, (360, 360))
                # break
                image_array = img.permute(1, 2, 0).numpy()
                image_array = (image_array * 255).astype('uint8')
                plt.imsave(f'{room_path}/{i}.png', image_array)
            
            n += 1

        if n >= num_vid:
            break
        # break
    # break

    return keys


In [33]:
# Takes around 3 min
# keys = save_images(path_to_torch, "/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_full")

Next cell is to create the image grids of ground truth and generated images

In [7]:
def make_img_grid(real_dir, gen_dir, output_dir):
    # with open(video_index_path, 'r') as file:
    #     video_index = json.load(file)

    for key_dir in tqdm(gen_dir.iterdir(), desc='Processing directories'):
        if not key_dir.suffix.lower() == '.json':
            key = key_dir.stem
            output_path = Path(output_dir + f"/{key}")
            output_path.mkdir(exist_ok=True, parents=True)

            # target_images = video_index[key]['target'] #[-46:] # Hardcoded for now

            for i, _ in enumerate((key_dir / Path("color")).iterdir()):
                # j = target_images[i]
                gen_img_path = key_dir / Path("color") / Path(f"{i:06d}.png")
                # real_img_path = Path(real_dir) / Path(key) / Path(str(j) + ".png")
                real_img_path = Path(real_dir) / Path(key) / Path("target") / Path(f"{i:06d}.png")

                fig, axes = plt.subplots(1, 2, figsize=(10, 5)) 

                # Display real image
                real_img = mpimg.imread(real_img_path)
                axes[0].imshow(real_img)
                axes[0].axis('off') 
                axes[0].set_title('Ground Truth')

                # Display generated image
                gen_img = mpimg.imread(gen_img_path)
                axes[1].imshow(gen_img)
                axes[1].axis('off')  
                axes[1].set_title('PixelSplat') # Hardcoded for now

                plt.subplots_adjust(wspace=0.05)
                plt.savefig(output_path / Path(f"{str(i)}.png"), bbox_inches='tight')
                plt.close(fig)

            # break for testing for just one room
            # break

In [9]:
# make_img_grid(gen_root, gen_root, "/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_img_grid")
make_img_grid(gen_root, gen_root, "../outputs/re10k_img_grid")

Processing directories: 1it [01:26, 86.84s/it]


And now create a video out of those image grids

In [4]:
def create_video(path_imgs, output_path, framerate=30):
    for room in Path(path_imgs).iterdir():  
        key = room.stem
        output_dir = Path(output_path)
        output_dir.mkdir(exist_ok=True, parents=True)


        img_array = []
        # for filename in glob.glob(f'{path_imgs}/{key}/*.png'):
        i = 0
        for _ in (Path(path_imgs)/Path(key)).iterdir():
            filename = path_imgs + "/" + key + f"/{i}.png"
            # print(filename)
            img = cv2.imread(filename)
            height, width, _ = img.shape
            size = (width,height)
            img_array.append(img)
            i += 1

        out = cv2.VideoWriter(f'{output_path}/{key}.avi',cv2.VideoWriter_fourcc(*'DIVX'), framerate, size)

        for i in range(len(img_array)):
            out.write(img_array[i])

        out.release()


In [ ]:
# create_video("/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_img_grid", 
#             "/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_video_grid")


create_video("../outputs/re10k_img_grid", 
            "../outputs/re10k_video_grid")

Now for ground truth, pixelsplat and instructir

In [ ]:
def make_img_grid(real_dir, gen_dir, output_dir):
    # with open(video_index_path, 'r') as file:
    #     video_index = json.load(file)

    for key_dir in tqdm(gen_dir.iterdir(), desc='Processing directories'):
        if not key_dir.suffix.lower() == '.json':
            key = key_dir.stem
            output_path = Path(output_dir + f"/{key}")
            output_path.mkdir(exist_ok=True, parents=True)

            # target_images = video_index[key]['target'] #[-46:] # Hardcoded for now

            for i, _ in enumerate((key_dir / Path("color")).iterdir()):
                # j = target_images[i]
                gen_img_path = key_dir / Path("color") / Path(f"{i:06d}.png")
                # real_img_path = Path(real_dir) / Path(key) / Path(str(j) + ".png")
                real_img_path = Path(real_dir) / Path(key) / Path("target") / Path(f"{i:06d}.png")
                inst_img_path = Path(real_dir) / Path(key) / Path("instructir") / Path(f"{i:06d}.png")                
                cont_img_path = Path(real_dir) / Path(key) / Path("controlnet") / Path(f"{i:06d}.png")

                fig, axes = plt.subplots(1, 4, figsize=(10, 5)) 

                # Display real image
                real_img = mpimg.imread(real_img_path)
                axes[0].imshow(real_img)
                axes[0].axis('off') 
                axes[0].set_title('Ground Truth')

                # Display generated image
                gen_img = mpimg.imread(gen_img_path)
                axes[1].imshow(gen_img)
                axes[1].axis('off')  
                axes[1].set_title('PixelSplat') # Hardcoded for now

                inst_img = mpimg.imread(inst_img_path)
                axes[2].imshow(inst_img)
                axes[2].axis('off')  
                axes[2].set_title('InstructIR')

                plt.subplots_adjust(wspace=0.05)
                plt.savefig(output_path / Path(f"{str(i)}.png"), bbox_inches='tight')
                plt.close(fig)

            # break for testing for just one room
            # break

In [5]:
ids = ['5aca87f95a9412c6', '20d86cff490c0c42', '41abd737e0228c1a', '1214f2a11a9fc1ed', '6558c5f10d45a929', '656381bea665bf3d', '3087828bc27bc4c7', 'c48f19e2ffa52523']
# c48f19e2ffa52523
ids = ['5aca87f95a9412c6']

ids = ['5aca87f95a9412c6', '20d86cff490c0c42', '41abd737e0228c1a', '1214f2a11a9fc1ed', '6558c5f10d45a929', '656381bea665bf3d', '3087828bc27bc4c7', 'c48f19e2ffa52523']


In [6]:
def make_img_grid(real_dir, gen_dir, output_dir):
    # with open(video_index_path, 'r') as file:
    #     video_index = json.load(file)
    a = False
    for key_dir in tqdm(gen_dir.iterdir(), desc='Processing directories'):
        # print(key_dir)
        if not key_dir.suffix.lower() == '.json':
            directory_path = key_dir / Path('controlnet2')
            # print(directory_path)
            if os.path.exists(directory_path) and key_dir.stem in ids:
                key = key_dir.stem
                output_path = Path(output_dir + f"/{key}")
                output_path.mkdir(exist_ok=True, parents=True)

                # target_images = video_index[key]['target'] #[-46:] # Hardcoded for now

                for i, _ in enumerate((key_dir / Path("color")).iterdir()):
                    # j = target_images[i]
                    gen_img_path = key_dir / Path("color") / Path(f"{i:06d}.png")
                    # real_img_path = Path(real_dir) / Path(key) / Path(str(j) + ".png")
                    real_img_path = Path(real_dir) / Path(key) / Path("target") / Path(f"{i:06d}.png")
                    inst_img_path = Path(real_dir) / Path(key) / Path("instructir2") / Path(f"{i:06d}.png")                
                    cont_img_path = Path(real_dir) / Path(key) / Path("controlnet2") / Path(f"{i:06d}.png")



                    fig, axes = plt.subplots(1, 4, figsize=(10, 5)) 

                    # Display real image
                    real_img = mpimg.imread(real_img_path)
                    axes[0].imshow(real_img)
                    axes[0].axis('off') 
                    axes[0].set_title('Ground Truth')

                    # Display generated image
                    gen_img = mpimg.imread(gen_img_path)
                    axes[1].imshow(gen_img)
                    axes[1].axis('off')  
                    axes[1].set_title('PixelSplat') # Hardcoded for now

                    inst_img = mpimg.imread(inst_img_path)
                    axes[2].imshow(inst_img)
                    axes[2].axis('off')  
                    axes[2].set_title('InstructIR')

                    cont_img = mpimg.imread(cont_img_path)
                    axes[3].imshow(cont_img)
                    axes[3].axis('off')  
                    axes[3].set_title('ControlNet')

                    plt.subplots_adjust(wspace=0.05)
                    plt.savefig(output_path / Path(f"{str(i)}.png"), bbox_inches='tight')
                    plt.close(fig)

            if key_dir.stem == '29e0bfbad00f0d5e':
                a = True

            # break for testing for just one room
            # break

In [7]:
gen_root = Path('/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_test_hard_more/re10k')

In [1]:
# 4 is with old instructir (Wouter01/instructir_hard_data, controlnet_conditioning_scale=1.0)
# 4.2 is with better instructir (Wouter01/InstructIR_re10k_hard, controlnet_conditioning_scale=1.0)
# 4.3 is with controlnet2 (Wouter01/InstructIR_re10k_hard, controlnet_conditioning_scale=2.0)
# 4.4 is with controlnet3 (Wouter01/InstructIR_re10k_hard, controlnet_conditioning_scale=4.0)

In [8]:
make_img_grid(gen_root, gen_root, "/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_img_grid_4.3")

Processing directories: 19it [06:06, 19.27s/it]


In [9]:
create_video("/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_img_grid_4.3", 
            "/teamspace/studios/this_studio/diffusion-augmented-pixelsplat/outputs/re10k_video_grid_4.3")